In [ ]:
using JuMP
using Gurobi
using TimerOutputs
to = TimerOutput()
N = 5000
A = rand((1:10),(2,N))

In [ ]:
start = time()
gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
m = Model(() -> Gurobi.Optimizer(gurobi_env))
@variable(m, y[1:N]>=0)
@objective(m, Min, y[1])

con1 = @constraint(m, A[1,1]* y[1] >= 0)
con2 = @constraint(m, A[2,1]* y[1] >= 0)
# println(m)
for t = 2:N
    set_normalized_coefficient(con1, y[t], A[1,t])
    set_normalized_coefficient(con2, y[t], A[2,t])
    @objective(m, Min, sum(y[l] for l = 1:t))
#     println(m)
    optimize!(m)
end
totalTime = time()-start
println(totalTime)

In [ ]:
start = time()
function MP(t,A)
    m = Model(() -> Gurobi.Optimizer(gurobi_env))
    @variable(m, y[1:t]>=0)
    @objective(m, Min, sum(y[l] for l = 1:t))
    con1 = @constraint(m, sum(A[1,l]* y[l] for l=1:t) >= 0)
    con2 = @constraint(m, sum(A[2,l]* y[l] for l=1:t) >= 0)
    optimize!(m)
end

for t = 1:N
    MP(t, A)
end
totalTime = time()-start
println(totalTime)

In [1]:
#Multiple Interdiction/Monitoring

using JuMP
using Gurobi
using TimerOutputs

global v = 10
global C = 2
global numPaths = 4
# include("./ConstructQD.jl")
global Q = [-C -C v v;
    v v -C v;
    v v v -C;
    -C v v v;
    v -C v v;
    -C v -C -C;
    v v v -C;
    -C -C v v;
    -C -C -C v;
    -C -C v -C;
    -C -C v v;
    -C -C v v;
    -C -C -C v;
    -C -C v -C;
    -C -C -C -C;
    v v -C -C;
    v v -C -C;
    -C v -C -C;
    v -C -C -C;
    -C v -C -C;
    v v -C -C]
global numMonitor = length(Q[:,1])

function SP(Q,y,sizeL,numMonitor)
    SP = Model(() -> Gurobi.Optimizer(gurobi_env))
    # @variable(m, y[1:numMonitor]>=0)
    @variable(SP, z[1:numPaths] >= 0)
    zcon = sum(z[1:sizeL] == 1)
    @objective(SP, Max, sum(sum(y[k]*Q[k,l] for k = 1:numMonitor)*z[l] for l = 1:sizeL))
    optimize!(SP)
    obj = JuMP.objective_value.(SP)
    z_now = JuMP.value.(z)
    dual = JuMP.dual(zcon)
    
    reducedCost = 0
    chosenPath = 0
    for l = (sizeL+1):numPaths
        if dual - sum(y[k]*Q[k,l] for k = 1:numMonitor) < reducedCost
            chosenPath = l
        end
    end
    return obj, z_now, dual, chosenPath
end
# function reducedCost(dual, Q, y, sizeL)
#     

SP (generic function with 1 method)

In [3]:

gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
m = Model(() -> Gurobi.Optimizer(gurobi_env))
@variable(m, y[1:numMonitor]>=0)
@variable(m, w >= -1e6)
# @variable(m, z[1:numPaths] >= 0)
@objective(m, Min, w)

for iter = 1:1
    global Q
    global numMonitor
#     @constraint(m, w)
    optimize!(m)
    obj = JuMP.objective_value.(m)
    y_now = JuMP.value.(y)
    obj, z_now, dual, chosenPath = SP(Q,y_now,iter,numMonitor)
end
# println(m)


ErrorException: Invalid Gurobi license